In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

Autosaving every 180 seconds


In [2]:
# LOAD the first 10,000 PREDICTIONS/CONFIDENCE MAPS THAT COME OUT OF DEEPLABCUT for a particular video
# Notes: 
#        Equivalent to 10000/25fps = 400seconds
#        Dimensionts: [# of Frames, width, height]
#        DLC PREDICTS ON 1280 x 1024 images, but they are downsampled to 128 x 160
#               i.e. if we need higher resolution, it can be obtained easily

predictions = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/predictions_10k_new_surfaces.npy')
print (predictions.shape)
plt.imshow(predictions[0])
plt.show()



(10000, 128, 160)


In [3]:
# LOAD THE FIRST 1,000 VIDEO FRAMES (there are 10k in the saved file)

import cv2
vidcap = cv2.VideoCapture('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi')
images = []
count=0
max_count = 1000 # THERE ARE 10,000 frames here;
while True:
    success, image = vidcap.read()
    
    if success==False:
        break
        
    images.append(image)
    if count%100==0:
        print (count)
    count += 1
    if count>=max_count:
        break
images = np.array(images)

0
100
200
300
400
500
600
700
800
900


In [99]:
# VISUALIZE AS SANITY CHECK SIDE BY SIDE
# SELECT FRAME

frame = 200

ax=plt.subplot(2,2,1)
plt.title("Raw video frame",fontsize=16)
plt.imshow(images[frame][::8,::8],alpha=1)
ax=plt.subplot(2,2,2)
plt.title("DLC (sum) heatmaps",fontsize=16)
plt.imshow(predictions[frame],alpha=1)
ax=plt.subplot(2,2,3)
plt.title("Overlayed as a sanity check",fontsize=16)
plt.imshow(images[frame][::8,::8],alpha=.5)
plt.imshow(predictions[frame],alpha=.5)
plt.show()


In [29]:
data = np.array(images).mean(3)
print (data.shape)

(1000, 1024, 1280)


In [30]:
median = np.median(data,axis=0)
plt.imshow(median)
plt.show()

In [54]:
# LOAD RANDOM 500 VIDEO FRAMES (there are 10k in the saved file)

import cv2
cap = cv2.VideoCapture('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi')

frames = np.random.choice(np.arange(89000), 500)
frames = np.sort(frames)
print (frames)

images = []
count=0
max_count = frames.shape[0] # THERE ARE 10,000 frames here;
while True:
    cap.set(cv2.CAP_PROP_POS_FRAMES, frames[count]) # optional
    success, image = cap.read()
    images.append(image)
    if count%25==0:
        print (count, frames[count])
    count += 1
    if count>=max_count:
        break


[  202   642   848   960  1139  1186  1342  1355  1656  1679  2445  2720
  2821  3035  3112  3163  3241  3467  3622  3653  4037  4211  4262  4476
  4768  5301  5756  5861  5966  6110  6333  7084  7227  7292  7691  7728
  7941  7989  8085  8174  8409  8422  8581  8768  9083  9208  9285  9435
  9467  9865 10179 10261 10415 10856 10958 11212 11227 11335 11739 11780
 12021 12262 12267 12592 12689 13021 13147 13159 13473 13478 13679 13683
 13790 14012 14164 14342 14537 14744 14799 14884 15170 15187 15226 15715
 15723 15787 16214 16368 16665 17288 17319 17966 18100 18245 18499 18656
 18663 18880 18925 19084 19139 19225 19381 19486 19559 19778 19985 20065
 20278 20908 21043 21728 21735 22107 22208 22478 22566 22702 22836 23271
 23454 23566 23796 23919 23959 24527 24935 25217 25223 25422 25482 25548
 25586 25642 25716 25894 26065 26347 26403 26437 26765 26914 26965 27018
 27026 27245 27437 27499 27518 27766 27793 28086 28270 28288 29688 29765
 30138 30194 30208 30587 30591 30899 30949 31486 31

In [63]:
data = np.float32(images)
print (data.shape)
#plt.imshow(data[30])
#plt.show()


(500, 1024, 1280, 3)


In [64]:
median = np.median(data,axis=0)
plt.suptitle("Median over 500 random frames from a 1 hr movie")
rgb = ['red','green','blue']
for k in range(3):
    ax=plt.subplot(2,2,k+1)
    plt.imshow(median[:,:,k])
    plt.title("Color channel: "+rgb[k])
    
plt.show()

In [86]:
fig=plt.figure(figsize=(20,20))
print (median.shape, data.shape)
ids = np.arange(0,36,1)
k=2
ctr=0
for id_ in ids:
    diff = median - data[id_]
    ax=plt.subplot(6,6,ctr+1)
    plt.title(" frame: "+str(frames[id_]))
    plt.imshow(diff[:,:,k])
    ctr+=1
plt.savefig('/home/cat/diff.png')
plt.close()
#plt.show()

(1024, 1280, 3) (500, 1024, 1280, 3)


In [67]:
print (data[0])

[[[14. 14. 47.]
  [14. 14. 47.]
  [14. 14. 47.]
  ...
  [13. 14. 39.]
  [13. 14. 39.]
  [13. 14. 39.]]

 [[14. 14. 47.]
  [14. 14. 47.]
  [14. 14. 47.]
  ...
  [13. 14. 39.]
  [13. 14. 39.]
  [13. 14. 39.]]

 [[14. 14. 47.]
  [14. 14. 47.]
  [14. 14. 47.]
  ...
  [13. 14. 39.]
  [13. 14. 39.]
  [13. 14. 39.]]

 ...

 [[30. 28. 61.]
  [30. 28. 61.]
  [30. 28. 61.]
  ...
  [12. 13. 31.]
  [12. 13. 31.]
  [12. 13. 31.]]

 [[30. 28. 61.]
  [30. 28. 61.]
  [30. 28. 61.]
  ...
  [12. 13. 31.]
  [12. 13. 31.]
  [12. 13. 31.]]

 [[30. 28. 61.]
  [30. 28. 61.]
  [30. 28. 61.]
  ...
  [12. 13. 31.]
  [12. 13. 31.]
  [12. 13. 31.]]]
